In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import datetime as dt
from xgboost import plot_importance, plot_tree
from auto_ts import auto_timeseries as ATS
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
plt.style.use('fivethirtyeight')

Imported auto_timeseries version:0.0.90. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



In [7]:
ruta_archivo = '/Users/macronald/U Central/Tesis Maestria Calidad/Insumos/df_TS.csv'
df = pd.read_csv(ruta_archivo)
df_check = df

In [8]:
train_df = df_check.iloc[:int(len(df_check) * 0.9)]
test_df = df_check.iloc[int(len(df_check) * 0.9):]
print(train_df.shape, test_df.shape)

(572162, 3) (63574, 3)


In [9]:
station_test = test_df.Estacion.unique()
station_test

array(['MinAmbiente', 'MovilFontibon', 'MovilSeptima', 'PuenteAranda',
       'SanCristobal', 'Suba', 'Tunal', 'Usaquen', 'Usme',
       'AltoRendimiento', 'Bolivia', 'CarvajalSevillana', 'CiudadBolivar',
       'Colina', 'Fontibon', 'Guaymaral', 'Jazmin', 'Kennedy',
       'LasFerias'], dtype=object)

In [23]:
train_df = train_df[train_df.Estacion == station_test[6]]
test_df = train_df[train_df.Estacion == station_test[6]]
test_df = test_df.drop('Estacion', axis=1)
train_df = train_df.drop('Estacion', axis=1)
test_df

,DateTime,ICA
11,2020-01-01 01:00:00,97.0
24,2020-01-01 02:00:00,104.0
37,2020-01-01 03:00:00,151.0
50,2020-01-01 04:00:00,128.0
63,2020-01-01 05:00:00,124.0
...,...,...
572073,2023-09-19 12:00:00,36.0
572092,2023-09-19 13:00:00,70.0
572111,2023-09-19 14:00:00,32.0
572130,2023-09-19 15:00:00,66.0
